In [3]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 11.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 8.9 MB/s eta 0:00:0000:01m00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 9.7 MB/s eta 0:00:00:00:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 10.4 MB/s eta 0:00:00
  Created wheel for validators:

In [8]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [10]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from dateutil.relativedelta import relativedelta
from datetime import datetime

# API URL
api_url = "https://api.eia.gov/v2/international/data/?api_key=PYTvNry19FP8DgMdBAikT14IOtP21vBOZY2ZuDTJ&frequency=monthly&data[0]=value&facets[activityId][]=1&facets[productId][]=53&facets[countryRegionId][]=IND&facets[unit][]=TBPD&start=1990-01&end=2023-03&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000"


In [14]:
import requests

In [15]:
#Read API data into a DataFrame
response = requests.get(api_url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data['response']['data'])
    df['ds'] = pd.to_datetime(df['period'])
    df['y'] = df['value']
    data_series = df[['ds', 'y']]

In [16]:

API_TOKEN = 'a122af78b08f7eccfe99605fe17fb2050d3116e1'

# Read CPI data from the specified URL
df_cpi = pd.read_csv(
    'https://www.econdb.com/api/series/CPIIN/?token=%s&format=csv' % API_TOKEN,
    parse_dates=['Date'])

# Merge the CPI dataframe and the data series dataframe
merged_df = df_cpi.merge(data_series, left_on='Date', right_on='ds', how='inner')

# Set the cutoff period for training
period = len(merged_df) - 6

# Select the training data
train_data = merged_df[['ds', 'CPIIN', 'y']][:period]

# Select the test data
test_data = merged_df[['ds', 'CPIIN', 'y']][period:]

# Define the ARIMA order
order = (1, 0, 0)

# Create and fit the ARIMAX model
model_arimax = ARIMA(train_data['y'], exog=train_data['CPIIN'], order=order)
model_arimax_fit = model_arimax.fit()


In [17]:


# Define the prediction function
def predict_oil_production(months_diff):
    # Predict CPIIN
    predicted_cpiin = model_fit.predict(start=len(data), end=len(data) + months_diff - 1)
    predicted_cpiin = pd.DataFrame(predicted_cpiin, columns=['CPIIN'])
    
    # Predict oil production using the trained ARIMAX model
    exog_reshaped = predicted_cpiin['CPIIN'].values.reshape(-1, 1)
    predicted_values = model_arimax_fit.predict(start=len(merged_df)-6, end=len(merged_df)-6 + months_diff - 1, exog=exog_reshaped)
    predicted_values = pd.DataFrame(predicted_values, columns=['Oil_Production'])
    
    # Generate monthly dates
    start_date = merged_df['ds'].dt.date.max() - relativedelta(months=5)
    end_date = datetime.today().date() + relativedelta(months=4)
    dates = pd.date_range(start=start_date, end=end_date, freq='MS')
    
    # Create a DataFrame with the predicted values and dates
    predicted_df = pd.DataFrame({'Month': dates})
    predicted_df = pd.concat([predicted_df, predicted_values], axis=1)
    
    return predicted_df

# Create Streamlit widgets
st.header('Oil Production Forecasting')

start_date = st.date_input('Select Test Start Date', value=merged_df['ds'].dt.date.max() - relativedelta(months=5))
end_date = st.date_input('Select Test End Date', value=datetime.today().date() + relativedelta(months=4))
button_predict = st.button('Predict')

# Define the main Streamlit app logic
if button_predict:
    date_diff = end_date - start_date
    months_diff = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
    
    # Predict for the specified duration
    st.subheader('Oil Production Forecast: Custom Date')
    prediction_custom = predict_oil_production(months_diff)
    st.dataframe(prediction_custom)
    
    # Predict for the short-term duration (6 months)
    st.subheader('Oil Production Forecast: Short-Term Duration')
    prediction_short_term = predict_oil_production(6)
    st.dataframe(prediction_short_term)
    
    # Predict for the long-term duration (60 months)
    st.subheader('Oil Production Forecast: Long-Term Duration')
    prediction_long_term = predict_oil_production(60)
    st.dataframe(prediction_long_term)
    
    # Reset the end date
    end_date = start_date + relativedelta(days=date_diff.days)
    
    # Plot the oil production forecast
    st.subheader('Oil Production Forecast: Graph')
    plt.figure(figsize=(8, 4))
    plt.plot(prediction_custom['Month'], prediction_custom['Oil_Production'], label='Custom Date')
    plt.plot(prediction_short_term['Month'], prediction_short_term['Oil_Production'], label='Short-Term Duration')
    plt.plot(prediction_long_term['Month'], prediction_long_term['Oil_Production'], label='Long-Term Duration')
    plt.xlabel('Month')
    plt.ylabel('Oil Production')
    plt.title('Oil Production Forecast')
    plt.legend()
    st.pyplot(plt)


2023-07-15 01:21:23.048 
  command:

    streamlit run /opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
